In [1]:
import torch
import numpy as np 
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt 
import os 
from torch.utils.data import Dataset,DataLoader,TensorDataset
import torch.autograd as autograd
from torch.autograd import Variable
from PIL import Image



In [2]:
from skimage import io
def getdata(dir):
    categories=['horizontal','vertical']
    im_data=[]
    labels=[]

    for c in categories:
        path=os.path.join(dir,c)
        for img in os.listdir(path):
            im_path=os.path.join(path,img)
            image=io.imread(im_path,as_gray=True)
            im_data.append(image)
            if(c=='horizontal'):
                labels.append(0)
            else :
                labels.append(1)
    
    im_data = np.asarray(im_data)
    labels = np.asarray(labels)
    labels = labels.reshape((1000,1))

    return im_data,labels        



In [3]:
dir = '/home/sreegs/Documents/CGAN/data'
images,labels=getdata(dir)

In [6]:
class train_data(Dataset):
    
    def __init__(self,labels,data,transform=None):
        self.transform = transform
        self.labels = labels
        self.images = data

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        label = self.labels[idx]
        img = Image.fromarray(self.images[idx])
        
        if self.transform:
            img = self.transform(img)

        return img, int(label)


In [8]:
transform=transforms.Compose([transforms.ToTensor,transforms.Normalize(mean=0.5,std=0.5)])
dataset= train_data(labels,images,transform=transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [13]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.label_emb = nn.Embedding(2, 2)
        self.model = nn.Sequential(
            nn.Linear(66, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x, labels):
        x = x.view(x.size(0), 64)
        c = self.label_emb(labels)
        x = torch.cat([x, c], 1)
        out = self.model(x)
        return out.squeeze()

In [14]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.label_emb = nn.Embedding(2, 2)
        
        self.model = nn.Sequential(
            nn.Linear(102, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 64),
            nn.Tanh()
        )
    
    def forward(self, z, labels):
        z = z.view(z.size(0), 100)
        c = self.label_emb(labels)
        x = torch.cat([z, c], 1)
        out = self.model(x)
        return out.view(x.size(0), 8, 8)

In [18]:
generator = Generator()
discriminator = Discriminator()
criterion = nn.BCELoss()
learning_rate=0.0001
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate)
